In [ ]:
import unicodedata # Normalización de texto
import re # Expresiones regulares
from langchain.text_splitter import RecursiveCharacterTextSplitter # División de texto
from sentence_transformers import SentenceTransformer, util # Modelos de embeddings y utilidades
from PyPDF2 import PdfReader # Lectura de PDFs
from pprint import pprint # Impresión formateada


In [ ]:
def clean_text(texto):
    texto = texto.lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8') # elimina acentos
    #texto = re.sub(r'[^a-z0-9\s]', '', texto)  # elimina puntuación
    texto = re.sub(r'\s+', ' ', texto).strip()  # espacios extra
    return texto


In [ ]:
def buscar_patron(texto, etiqueta, patron):
    match = re.search(patron, texto, re.IGNORECASE)
    if match:
        return f"{etiqueta.upper()}: {match.group(0).strip()}"
    else:
        return f"{etiqueta.upper()}: No disponible"

def extract_metadata(texto):
    metadata = {}
    metadata['csj'] = buscar_patron(texto, 'csj', r'CSJ\s*[^\n\.]+')
    metadata['corte'] = buscar_patron(texto, 'corte', r'CORTE\s*[^\n\.]+')
    metadata['provincia'] = buscar_patron(texto, 'provincia', r'BUENOS AIRES\s*[^\n\.]+')
    metadata['firma'] = buscar_patron(texto, 'firma', r'FIRMADO\s*[^\n\.]+')
    metadata['parte_actora'] = buscar_patron(texto, 'parte_actora', r'PARTE ACTORA\s*[^\n\.]+')
    metadata['parte_demandada'] = buscar_patron(texto, 'parte_demandada', r'PARTE DEMANDADA\s*[^\n\.]+')
    return metadata


In [ ]:
def chunk_text(texto):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    chunks = splitter.split_text(texto)
    return [
        {
            "original_text": chunk,
            "cleaned_text": clean_text(chunk)
        }
        for chunk in chunks
    ]


In [ ]:
modelo = SentenceTransformer('all-MiniLM-L6-v2')
def generate_embeddings(corpus):
    """
    Genera embeddings para un corpus de texto.
    corpus: {"original_text": str, "cleaned_text": str}
    """
    original_text = [chunk["original_text"] for chunk in corpus]
    return modelo.encode(original_text, convert_to_tensor=True, device='cpu')

def generate_embeddings_str(string):
    """
    Genera embeddings para un string de texto.
    """
    return modelo.encode(string), 


def query_similarity(consulta, embeddings_corpus):
    """
    Calcula la similitud entre una consulta y un corpus de texto.
    """
    embedding_consulta = generate_embeddings_str(consulta)
    
    scores = util.cos_sim(embedding_consulta, embeddings_corpus)
    
    return scores



/home/manuel/research/lawAI/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def extract_text_from_pdf(pdf_path): 
    reader = PdfReader(pdf_path)
    texto = ""
    for page in reader.pages:
        texto += page.extract_text() + "\n"
    return texto.strip()

def search_sentence_in_pdf(pdf_path, sentence):
    reader = PdfReader(pdf_path)
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        if text and sentence.strip() in text:
            return i + 1  # Página donde aparece
    return None

In [ ]:

# main
if __name__ == "__main__":
    pdf_path = 'falloEj2.pdf'
    texto=""
    """ with open('falloEj2.pdf', 'r', encoding='utf-8') as archivo:
        texto = archivo.read() """
    texto = extract_text_from_pdf(pdf_path)
    texto_limpio = clean_text(texto)
    metadatos = extract_metadata(texto_limpio)
    chunks = chunk_text(texto)
    corpus = chunks  # usamos los fragmentos como corpus
    embeddings_corpus = generate_embeddings(corpus)
    consulta = "notificaciones"
    scores = query_similarity(consulta, embeddings_corpus)
    for i, score in enumerate(scores[0]):
        print(f"similitud: {score.item():.2f} (página {search_sentence_in_pdf(pdf_path, corpus[i]['original_text'])}) -> {corpus[i]['cleaned_text']} ")
    
        
        

similitud: 0.29 (página 1) -> csj 6066/2014/rh1 y otros molina, luis omar y otros c/ estado nacional - ministerio de justicia, seguridad social y derechos humanos y otro s/ personal militar y civil de las ffaa y de seg. corte suprema de justicia de la nacion 1vistos los autos: recursos de hecho deducidos por la actora en la causa 'molina, luis omar y otros c/ estado nacional - ministerio 
similitud: 0.27 (página 1) -> de justicia seguridad social y derechos humanos y otro s/ personal militar y civil de las ffaa y de seg.'; css 55491/2010/1/rh1 'aranda norma nelida y otros c/ min justicia seg y dd hh y otro s/ personal militar y civil de las ffaa y de seg.'; css 68528/2010/1/rh1 'benitez damian y otros c/ min justicia seg y dd hh y otro s/ personal militar 
similitud: 0.31 (página 1) -> y civil de las ffaa y de seg; css 81217/2011/1/rh1 'mora timoteo y otros c/ ministerio de seguridad y otro s/ , para decidir personal militar y civil de las ffaa y de seg.'" sobre su procedencia. conside

/home/manuel/research/lawAI/venv/lib/python3.11/site-packages/sentence_transformers/util.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  a = torch.tensor(a)
